# Structured Outputs : Sorties JSON Garanties avec OpenAI

Dans ce notebook, nous explorons les **Structured Outputs**, une fonctionnalité qui garantit que le modèle retourne des données JSON valides selon un schéma prédéfini.

**Objectifs :**
- Comprendre la différence entre JSON mode et Structured Outputs
- Maîtriser `response_format` avec JSON Schema
- Intégrer Pydantic pour une validation type-safe
- Gérer les erreurs de schéma

**Prérequis :** Notebook 1 (OpenAI Intro)

**Durée estimée :** 55 minutes

# 3. Structured Outputs : Sorties JSON Garanties

**Navigation** : [<< Precedent](2_PromptEngineering.ipynb) | [Index](../../README.md) | [Suivant >>](4_Function_Calling.ipynb)

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Comprendre la difference entre JSON mode et Structured Outputs
2. Maîtriser response_format avec JSON Schema
3. Integrer Pydantic pour une validation type-safe
4. Gérer les erreurs de schema

### Prerequis
- Notebook 1 (OpenAI Intro)
- Python 3.10+
- Cle API OpenAI configuree

### Duree estimee : 55 minutes

---

# Structured Outputs : Sorties JSON Garanties avec OpenAI

In [1]:
# Installation des dépendances et configuration
%pip install -q openai python-dotenv pydantic

import os
import json
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List, Optional

load_dotenv('../.env')
client = OpenAI()

# Charger le modèle depuis .env ou utiliser gpt-5-mini par défaut
DEFAULT_MODEL = os.getenv("OPENAI_MODEL", "gpt-5-mini")
BATCH_MODE = os.getenv("BATCH_MODE", "false").lower() == "true"

def add_additional_properties_false(schema: dict) -> dict:
    """
    Ajoute récursivement additionalProperties: False à tous les objets du schema
    ET s'assure que tous les champs sont dans required (requis par OpenAI).
    
    OpenAI Structured Outputs exige:
    1. additionalProperties: False sur tous les objets
    2. Tous les champs dans 'required' (même ceux avec defaults)
    """
    if not isinstance(schema, dict):
        return schema
    
    # Traiter les définitions $defs d'abord (Pydantic 2.x les génère)
    if "$defs" in schema:
        for def_name, def_schema in schema["$defs"].items():
            add_additional_properties_false(def_schema)
    
    # Si c'est un objet, ajouter additionalProperties: False
    # ET s'assurer que tous les champs sont dans required
    if schema.get("type") == "object":
        schema["additionalProperties"] = False
        # Ajouter tous les champs de properties dans required
        if "properties" in schema:
            all_props = list(schema["properties"].keys())
            schema["required"] = all_props
            # Traiter récursivement les propriétés
            for prop_name, prop_schema in schema["properties"].items():
                add_additional_properties_false(prop_schema)
    
    # Traiter les items d'arrays
    if schema.get("type") == "array" and "items" in schema:
        add_additional_properties_false(schema["items"])
    
    # Traiter anyOf, allOf, oneOf
    for key in ["anyOf", "allOf", "oneOf"]:
        if key in schema:
            for item in schema[key]:
                add_additional_properties_false(item)
    
    return schema

print("Client OpenAI initialisé !")
print(f"Modèle par défaut: {DEFAULT_MODEL}")

## 1. JSON Mode vs Structured Outputs

OpenAI propose deux approches pour obtenir des sorties JSON :

### JSON Mode
- Garantit une **sortie JSON valide** (pas de syntaxe invalide)
- **NE garantit PAS** la structure exacte du JSON
- Le modèle peut inventer des clés, omettre des champs, etc.
- Utilise `response_format={"type": "json_object"}`

### Structured Outputs
- Garantit une sortie JSON **valide ET conforme au schéma**
- Le modèle respecte strictement le JSON Schema fourni
- Pas de champs manquants, pas de types incorrects
- Utilise `response_format={"type": "json_schema", "json_schema": {...}}`
- Disponible depuis août 2024 (gpt-4o-2024-08-06 et versions ultérieures)

**Cas d'usage :**
- JSON Mode : expérimentation rapide, structure flexible
- Structured Outputs : production, insertion base de données, APIs strictes

### Démonstration : Limites du JSON Mode

Avant de voir les Structured Outputs, expérimentons avec le JSON Mode classique pour comprendre ses limites. Le modèle va générer du JSON valide, mais **sans garantie sur la structure** : les noms de clés, le niveau d'imbrication, et l'organisation des données sont laissés à sa discrétion.

In [2]:
# JSON mode basique (ancienne approche)
response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[{"role": "user", "content": "Liste 3 fruits avec leurs couleurs en JSON"}],
    response_format={"type": "json_object"}
)

result = json.loads(response.choices[0].message.content)
print("JSON Mode - Structure non garantie :")
print(json.dumps(result, indent=2, ensure_ascii=False))
print("\nNote : Le modèle a choisi la structure librement (clé 'fruits', sous-objets, noms de clés)")

### Observation du JSON Mode

Le modèle a généré un JSON **syntaxiquement valide**, mais **structure librement choisie** :

**Analyse de la sortie** :

| Élément | Décision du modèle | Problème potentiel |
|---------|-------------------|-------------------|
| Clé racine | `"fruits"` | Pourrait être `"items"`, `"list"`, `"data"`, etc. Impossible à prédire |
| Structure items | Array d'objets avec `name`/`color` | Pourrait être `{"Banane": "Jaune", ...}` (dict) ou `[["Banane", "Jaune"], ...]` (tuples) |
| Noms de clés | `name`, `color` (anglais) | Le prompt était en français, le modèle a choisi l'anglais |
| Nombre de fruits | 3 (conforme à la demande) | ✅ Respecté |

**Risques en production** :

Si vous parsez ce JSON avec du code rigide :
```python
for fruit in result["fruits"]:  # KeyError si le modèle choisit "items"
    print(fruit["name"])  # KeyError si le modèle choisit "nom"
```

Le code peut **casser aléatoirement** selon l'humeur du modèle ou des variations du prompt.

**Cas d'usage acceptable pour JSON Mode** :
- Prototypage rapide où la structure exacte importe peu
- Affichage direct à l'utilisateur sans parsing
- Logging/debug

**Quand passer aux Structured Outputs** :
- Insertion en base de données (schéma strict requis)
- APIs avec contrats (clients attendent des champs précis)
- Workflows multi-étapes (sortie d'une étape = entrée de la suivante)

### Premier Exemple : Schéma Simple pour une Recette

Créons maintenant un schéma JSON strict pour forcer le modèle à respecter une structure précise. Ce premier exemple illustre les éléments essentiels d'un JSON Schema :

- **`type`** : Type de données (`object`, `array`, `string`, etc.)
- **`properties`** : Définition des champs de l'objet
- **`required`** : Liste des champs obligatoires
- **`additionalProperties: false`** : Interdit les champs non définis (requis par OpenAI)
- **`enum`** : Restriction à un ensemble de valeurs autorisées

**Note importante** : OpenAI exige que `additionalProperties` soit `false` à **tous les niveaux** du schéma pour activer le mode strict.

## 2. Introduction aux Structured Outputs

Les Structured Outputs utilisent **JSON Schema** pour définir précisément la structure attendue.

**Syntaxe :**
```python
response_format={
    "type": "json_schema",
    "json_schema": {
        "name": "nom_du_schema",
        "strict": True,  # Mode strict obligatoire
        "schema": {       # JSON Schema
            "type": "object",
            "properties": {...},
            "required": [...],
            "additionalProperties": False
        }
    }
}
```

**Points clés :**
- `strict: True` est **obligatoire** pour les Structured Outputs
- `additionalProperties: False` empêche les champs non définis
- Tous les champs obligatoires doivent être dans `required`
- Types supportés : `string`, `number`, `integer`, `boolean`, `array`, `object`, `enum`

### Exemple avec Objets Imbriqués

Passons à un modèle plus complexe avec des **objets imbriqués** (`Ingredient`) et des **listes typées**. Pydantic gère automatiquement la génération du JSON Schema pour ces structures complexes.

**Points techniques** :
- `List[Ingredient]` génère un schéma pour un tableau d'objets structurés
- `Field(description=...)` enrichit le schéma avec des descriptions que le modèle utilise comme contexte
- La fonction `add_additional_properties_false()` est nécessaire car Pydantic ne génère pas cette propriété par défaut

Cette approche est beaucoup plus **maintenable** et **type-safe** que l'écriture manuelle de JSON Schema.

In [3]:
# Structured Output avec schéma manuel
schema = {
    "type": "object",
    "properties": {
        "nom": {"type": "string"},
        "ingredients": {"type": "array", "items": {"type": "string"}},
        "temps_minutes": {"type": "integer"},
        "difficulte": {"type": "string", "enum": ["facile", "moyen", "difficile"]}
    },
    "required": ["nom", "ingredients", "temps_minutes", "difficulte"],
    "additionalProperties": False
}

response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[{"role": "user", "content": "Donne-moi une recette de crêpes."}],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "recette_schema",
            "strict": True,
            "schema": schema
        }
    }
)

recette = json.loads(response.choices[0].message.content)
print("Structured Output - Structure garantie :\n")
print(f"Recette: {recette['nom']}")
print(f"Difficulté: {recette['difficulte']}")
print(f"Temps: {recette['temps_minutes']} minutes")
print(f"Ingrédients: {', '.join(recette['ingredients'])}")

### Analyse du résultat

Comparons les sorties JSON Mode (cellule précédente) vs Structured Outputs :

**Différences clés** :

| Aspect | JSON Mode (cellule 4) | Structured Outputs (cellule 8) |
|--------|----------------------|-------------------------------|
| **Clé racine** | `"fruits"` (inventée) | `"nom"`, `"ingredients"`, etc. (imposées) |
| **Noms de champs** | `"name"`, `"color"` (anglais, au choix du modèle) | Exactement ceux du schéma (français) |
| **Champs garantis** | ❌ (le modèle peut omettre `temps_minutes`) | ✅ Tous les champs de `required` présents |
| **Type `difficulte`** | ❌ Pourrait être "simple" ou "easy" | ✅ Limité à `["facile", "moyen", "difficile"]` via `enum` |
| **Validation automatique** | ❌ Nécessite parsing manuel | ✅ JSON garanti conforme au schéma |

**Résultat obtenu** :
- **Nom** : "Crêpes Classiques"
- **Difficulté** : "facile" (conforme à l'enum)
- **Temps** : 30 minutes (type `integer` validé)
- **Ingrédients** : Liste de strings (structure exacte du schéma)

**Implications pratiques** :

Sans Structured Outputs, vous devriez écrire :
```python
if "nom" not in recette:
    raise ValueError("Champ 'nom' manquant")
if recette["difficulte"] not in ["facile", "moyen", "difficile"]:
    raise ValueError("Difficulté invalide")
# ... et ainsi de suite pour chaque champ
```

Avec Structured Outputs, **tout cela est géré automatiquement par OpenAI**. Le modèle ne peut physiquement pas retourner un JSON non conforme.

### Exemple Réaliste : Extraction de CV

L'extraction de données structurées depuis du texte non structuré est l'un des cas d'usage les plus puissants des Structured Outputs. 

**Défi technique** : Le mode strict d'OpenAI impose que **tous les champs soient dans `required`**. Pour simuler des champs optionnels (comme `email` ou `telephone` qui peuvent être absents d'un CV), nous utilisons des **valeurs par défaut** (`""` pour les strings) au lieu de `Optional[str]`.

Cette approche permet de :
- Garantir que le schéma est conforme aux exigences OpenAI
- Gérer l'absence de certaines informations avec des valeurs par défaut
- Maintenir la type safety Pydantic

**Résultat attendu** : Le modèle va parser le texte du CV et remplir automatiquement toutes les structures (expériences, formations, compétences) en respectant strictement le schéma Pydantic.

### Fonction Utilitaire Réutilisable

Pour éviter de répéter le même code de gestion d'erreurs et de génération de schéma, créons une **fonction générique** qui :

1. Accepte n'importe quel modèle Pydantic
2. Génère le JSON Schema adapté
3. Gère les erreurs API et de validation
4. Retourne une instance validée ou `None`

Cette fonction peut ensuite être réutilisée pour tous les cas d'extraction structurée du projet.

**Test de la fonction** : Évaluons un texte simple avec un modèle incluant des contraintes de validation (`conint(ge=0, le=100)` pour forcer un score entre 0 et 100).

## 3. Intégration avec Pydantic

Écrire des JSON Schemas manuellement est verbeux et source d'erreurs. **Pydantic** simplifie cela :

**Avantages :**
- **Type safety** : Validation automatique des types Python
- **Auto-complétion** : IDE détecte les champs et types
- **Génération automatique** : `model_json_schema()` crée le JSON Schema
- **Validation bidirectionnelle** : `model_validate_json()` vérifie et convertit
- **Documentation** : `Field(description=...)` enrichit le schéma

**Workflow :**
1. Définir les modèles Pydantic avec `BaseModel`
2. Générer le JSON Schema avec `.model_json_schema()`
3. Envoyer à l'API OpenAI
4. Valider la réponse avec `.model_validate_json()`

**Simulation de feedback client** : Le modèle va analyser le texte, identifier automatiquement :
- Le **sentiment** (ici mixte : positif sur l'appli, négatif sur le bug)
- La **catégorie** (probablement `bug_report`)
- La **priorité** (haute car bloquant pour le travail)
- Les bugs et suggestions implicites

In [4]:
# Pydantic avec model_json_schema()
class Ingredient(BaseModel):
    nom: str = Field(description="Nom de l'ingrédient")
    quantite: str = Field(description="Quantité nécessaire")

class RecetteStructuree(BaseModel):
    nom: str = Field(description="Nom de la recette")
    description: str = Field(description="Description courte")
    ingredients: List[Ingredient] = Field(description="Liste des ingrédients")
    etapes: List[str] = Field(description="Étapes de préparation")
    temps_preparation: int = Field(description="Temps en minutes")
    difficulte: str = Field(description="Niveau de difficulté")

# Générer le schema et ajouter additionalProperties: False
schema = RecetteStructuree.model_json_schema()
schema = add_additional_properties_false(schema)

response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[{"role": "user", "content": "Donne-moi une recette de mousse au chocolat détaillée."}],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "recette_detaillee",
            "strict": True,
            "schema": schema
        }
    }
)

# Validation automatique avec Pydantic
recette = RecetteStructuree.model_validate_json(response.choices[0].message.content)
print(f"Recette: {recette.nom}")
print(f"Temps: {recette.temps_preparation} min | Difficulté: {recette.difficulte}")
print(f"\nDescription:\n{recette.description}")
print("\nIngrédients:")
for ing in recette.ingredients:
    print(f"  - {ing.nom}: {ing.quantite}")
print("\nÉtapes:")
for i, etape in enumerate(recette.etapes, 1):
    print(f"  {i}. {etape}")

### Analyse du résultat

Le modèle a généré une recette complète et structurée en respectant parfaitement le schéma Pydantic :

**Structure validée** :

| Champ | Type | Exemple | Validation |
|-------|------|---------|------------|
| `nom` | `str` | "Mousse au Chocolat Délicieuse" | Non vide |
| `description` | `str` | "Une mousse aérienne..." | Description concise |
| `ingredients` | `List[Ingredient]` | 5 ingrédients avec quantités | Objets imbriqués |
| `etapes` | `List[str]` | 8 étapes numérotées | Séquence logique |
| `temps_preparation` | `int` | 20 minutes | Entier positif |
| `difficulte` | `str` | "Facile" | Enum implicite |

**Avantages de Pydantic détectés** :

1. **Auto-complétion IDE** : `recette.nom`, `recette.ingredients[0].quantite` → détection automatique des champs
2. **Type safety** : `recette.temps_preparation` est garanti être un `int`, pas une string
3. **Validation** : Si le modèle retournait `temps_preparation: "vingt"`, Pydantic lèverait une `ValidationError`
4. **Itération sécurisée** : `for ing in recette.ingredients` → chaque `ing` est un objet `Ingredient` typé

**Comparaison avec JSON manuel** :

```python
# Sans Pydantic (fragile)
recette_json = json.loads(response.content)
print(recette_json["nom"])  # Risque KeyError si clé manquante
temps = int(recette_json["temps"])  # Risque ValueError si type incorrect

# Avec Pydantic (robuste)
recette = RecetteStructuree.model_validate_json(response.content)
print(recette.nom)  # Garanti exister et être une string
temps = recette.temps_preparation  # Garanti être un int
```

> **Note** : Le schéma Pydantic sert de **contrat** entre l'API et le code. Toute modification du modèle sera détectée par le type checker (mypy, Pylance).

## 4. Cas d'Usage Avancés

Les Structured Outputs excellent dans les scénarios nécessitant une **structure de données fiable** :

### Extraction de Données
- **Documents** : CVs, factures, contrats
- **Emails** : Extraction expéditeur, sujet, actions requises
- **Articles** : Métadonnées (titre, auteur, date, mots-clés)

### Génération de Formulaires
- **Questionnaires dynamiques** : Générer des questions avec types de réponse
- **Configurations** : Paramètres applicatifs structurés

### Intégrations API
- **Réponses normalisées** : Format uniforme pour frontend
- **Insertion base de données** : Validation avant INSERT/UPDATE
- **Webhooks** : Payload structuré pour intégrations tierces

### Workflows Multi-Étapes
- **Agents** : Chaque étape retourne une structure validée
- **Pipelines** : Chaînage de transformations type-safe

In [5]:
# Extraction structurée de CV
# Note: Pour Structured Outputs strict, tous les champs doivent être dans required.
# Les champs peuvent avoir des valeurs par défaut pour être "optionnels" en pratique.

class Experience(BaseModel):
    entreprise: str
    poste: str
    dates: str
    description: str = ""  # Valeur par défaut au lieu d'Optional

class Formation(BaseModel):
    etablissement: str
    diplome: str
    annee: int

class CVStructure(BaseModel):
    nom: str
    email: str = ""  # Valeur par défaut
    telephone: str = ""  # Valeur par défaut
    experiences: List[Experience]
    formations: List[Formation]
    competences: List[str]

texte_cv = """
Jean Dupont - jean.dupont@email.com - 06 12 34 56 78

EXPÉRIENCES:
- Développeur Senior chez TechCorp (2020-2023): Développement d'applications Python
- Développeur Junior chez StartupXYZ (2018-2020): Maintenance de bases de données

FORMATION:
- Master Informatique, Université Paris-Saclay, 2018
- Licence Informatique, Université Lyon 1, 2016

COMPÉTENCES: Python, SQL, Docker, Git, Machine Learning
"""

# Générer le schema et ajouter additionalProperties: False
schema = CVStructure.model_json_schema()
schema = add_additional_properties_false(schema)

response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[{"role": "user", "content": f"Extrais les informations de ce CV:\n\n{texte_cv}"}],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "cv_extraction",
            "strict": True,
            "schema": schema
        }
    }
)

cv = CVStructure.model_validate_json(response.choices[0].message.content)
print("=== CV STRUCTURÉ ===\n")
print(f"Candidat: {cv.nom}")
print(f"Contact: {cv.email} | {cv.telephone}")
print(f"\nExpériences ({len(cv.experiences)}):")
for exp in cv.experiences:
    print(f"  - {exp.poste} @ {exp.entreprise} ({exp.dates})")
    if exp.description:
        print(f"    -> {exp.description}")
print(f"\nFormations ({len(cv.formations)}):")
for form in cv.formations:
    print(f"  - {form.diplome} ({form.etablissement}, {form.annee})")
print(f"\nCompétences: {', '.join(cv.competences)}")

### Analyse de l'extraction structurée

Le modèle a parfaitement parsé le CV non structuré en objets Pydantic typés :

**Données extraites avec succès** :

| Catégorie | Résultat | Précision |
|-----------|----------|-----------|
| Identité | Nom, email, téléphone | 100% |
| Expériences | 2 postes avec dates et descriptions | Chronologie respectée |
| Formations | 2 diplômes avec établissements et années | Ordre anti-chronologique |
| Compétences | 5 compétences techniques | Liste complète |

**Points techniques importants** :

1. **Gestion des champs "optionnels"** : Les champs `email`, `telephone`, `description` ont des valeurs par défaut `""` au lieu d'être `Optional[str]`. Cela respecte l'exigence OpenAI que **tous les champs soient dans `required`** en mode strict.

2. **Objets imbriqués** : `List[Experience]` et `List[Formation]` sont correctement gérés. Le JSON Schema généré par Pydantic inclut des définitions d'objets imbriqués (`$defs`).

3. **Validation automatique** : `CVStructure.model_validate_json()` garantit que chaque expérience a tous les champs requis et que les années sont bien des entiers.

**Cas d'usage production** :
- ATS (Applicant Tracking System) : Parsing automatique de milliers de CVs
- Matching candidat-poste : Extraction des compétences pour scoring
- Anonymisation : Retrait automatique des données personnelles avant analyse RH
- Base de données talents : Insertion directe en base avec schéma validé

## 5. Gestion des Erreurs

Même avec Structured Outputs, plusieurs types d'erreurs peuvent survenir :

### Erreurs API
- **Quota dépassé** : Limite de tokens ou requêtes
- **Modèle non supporté** : Structured Outputs requiert gpt-4o-2024-08-06+
- **Schéma invalide** : JSON Schema mal formé ou trop complexe

### Erreurs de Validation Pydantic
- **Type mismatch** : Le modèle retourne un type incorrect (rare avec strict mode)
- **Contraintes non respectées** : `Field(ge=0, le=100)` violé
- **Champs manquants** : Champ requis absent (ne devrait pas arriver)

### Bonnes Pratiques
- **Try/except** : Toujours wrapper les appels API
- **Validation explicite** : Utiliser `model_validate_json()` plutôt que `json.loads()`
- **Logging** : Logger les erreurs pour debug
- **Fallback** : Prévoir une stratégie de secours (retry, valeur par défaut)

In [6]:
# Gestion robuste des erreurs
from pydantic import ValidationError, conint

def extract_structured(prompt: str, model_class: type[BaseModel], model: str = None):
    """
    Fonction utilitaire pour extraction structurée avec gestion d'erreurs.

    Args:
        prompt: Prompt utilisateur
        model_class: Classe Pydantic pour validation
        model: Modèle OpenAI à utiliser (défaut: DEFAULT_MODEL)

    Returns:
        Instance validée de model_class ou None en cas d'erreur
    """
    if model is None:
        model = DEFAULT_MODEL
    
    try:
        # Générer le schema et ajouter additionalProperties: False (requis par OpenAI)
        schema = model_class.model_json_schema()
        schema = add_additional_properties_false(schema)

        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": model_class.__name__.lower(),
                    "strict": True,
                    "schema": schema
                }
            }
        )
        return model_class.model_validate_json(response.choices[0].message.content)
    except ValidationError as e:
        print(f"Erreur de validation Pydantic: {e}")
        return None
    except Exception as e:
        print(f"Erreur API: {e}")
        return None

# Test de la fonction utilitaire
class Evaluation(BaseModel):
    score: conint(ge=0, le=100) = Field(description="Score de 0 à 100")
    commentaire: str = Field(description="Commentaire détaillé")
    points_forts: List[str] = Field(description="Points forts identifiés")
    points_amelioration: List[str] = Field(description="Points à améliorer")

eval_result = extract_structured(
    "Évalue ce texte: 'Python est un langage de programmation populaire.'",
    Evaluation
)

if eval_result:
    print("=== ÉVALUATION STRUCTURÉE ===\n")
    print(f"Score: {eval_result.score}/100")
    print(f"Commentaire: {eval_result.commentaire}")
    print("\nPoints forts:")
    for pf in eval_result.points_forts:
        print(f"  + {pf}")
    print("\nPoints d'amélioration:")
    for pa in eval_result.points_amelioration:
        print(f"  - {pa}")
else:
    print("Échec de l'extraction structurée")

### Interprétation des résultats

L'évaluation structurée démontre la puissance de la validation Pydantic combinée aux Structured Outputs :

**Résultats obtenus** :
- **Score** : 85/100 (validé automatiquement dans la plage 0-100 grâce à `conint`)
- **Analyse qualitative** : Le modèle a identifié que le texte est factuel mais manque de détails
- **Points forts** : Concision, clarté de l'affirmation
- **Axes d'amélioration** : Enrichissement avec exemples et justifications

**Validation robuste** :

La fonction `extract_structured()` encapsule les bonnes pratiques :

1. **Génération automatique du schéma** : `model_json_schema()` évite l'écriture manuelle
2. **Ajout de `additionalProperties: false`** : Requis par OpenAI strict mode
3. **Gestion des erreurs API** : Timeouts, quotas, modèles invalides
4. **Validation Pydantic** : `model_validate_json()` garantit type safety
5. **Retour gracieux** : `None` en cas d'erreur au lieu d'exception non gérée

> **Avantage production** : Cette fonction peut être réutilisée pour tous les cas d'extraction structurée du projet. Créer une librairie partagée avec des modèles Pydantic communs (feedback, facture, événement, etc.) accélère le développement.

## 6. Exemple Complet : Analyseur de Feedback Produit

Cas d'usage réaliste : analyser des feedbacks clients et les structurer pour analyse.

**Objectifs :**
- Extraire sentiment, catégorie, priorité
- Identifier les fonctionnalités mentionnées
- Détecter les bugs signalés
- Structurer pour insertion en base de données

In [7]:
# Analyseur de feedback produit
class Bug(BaseModel):
    description: str = Field(description="Description du bug")
    severite: str = Field(description="Sévérité: critique, haute, moyenne, basse")

class FeedbackAnalyse(BaseModel):
    sentiment: str = Field(description="Sentiment général: positif, négatif, neutre, mixte")
    categorie: str = Field(description="Catégorie: feature_request, bug_report, question, praise, complaint")
    priorite: str = Field(description="Priorité: haute, moyenne, basse")
    resume: str = Field(description="Résumé en une phrase")
    fonctionnalites_mentionnees: List[str] = Field(description="Fonctionnalités citées")
    bugs_detectes: List[str] = Field(description="Bugs identifiés")
    suggestions: List[str] = Field(description="Suggestions d'amélioration")
    actions_recommandees: List[str] = Field(description="Actions à entreprendre")

feedback_client = """
J'adore l'application en général, mais le bouton d'export ne fonctionne pas depuis la dernière mise à jour.
J'ai essayé plusieurs fois et rien ne se passe. C'est vraiment frustrant car j'en ai besoin pour mon travail.
Sinon l'interface mobile pourrait être améliorée, elle est un peu confuse.
"""

analyse = extract_structured(
    f"Analyse ce feedback client et structure-le:\n\n{feedback_client}",
    FeedbackAnalyse
)

if analyse:
    print("=== ANALYSE DE FEEDBACK ===\n")
    print(f"Sentiment: {analyse.sentiment}")
    print(f"Catégorie: {analyse.categorie}")
    print(f"Priorité: {analyse.priorite}")
    print(f"\nRésumé:\n{analyse.resume}")
    print(f"\nFonctionnalités mentionnées:")
    for feat in analyse.fonctionnalites_mentionnees:
        print(f"  • {feat}")
    if analyse.bugs_detectes:
        print(f"\nBugs détectés:")
        for bug in analyse.bugs_detectes:
            print(f"  🐛 {bug}")
    if analyse.suggestions:
        print(f"\nSuggestions:")
        for sug in analyse.suggestions:
            print(f"  💡 {sug}")
    print(f"\nActions recommandées:")
    for action in analyse.actions_recommandees:
        print(f"  → {action}")

### Interprétation de l'analyse de feedback

Le modèle a correctement identifié :

| Dimension | Valeur détectée | Justification |
|-----------|----------------|---------------|
| **Sentiment** | Mixte | "J'adore" (positif) vs "frustrant" (négatif) |
| **Catégorie** | `bug_report` | Dysfonctionnement signalé explicitement |
| **Priorité** | Haute | Impact métier direct ("besoin pour mon travail") |
| **Bug principal** | Bouton export non fonctionnel | Depuis dernière mise à jour |
| **Suggestion** | Amélioration interface mobile | Point secondaire mais identifié |

**Actions extraites automatiquement** :
1. Corriger le bug d'export (prioritaire)
2. Planifier amélioration UX mobile

Ce type de structuration permet :
- Tri automatique par priorité dans un système de ticketing
- Calcul de métriques (distribution sentiments, catégories)
- Routing intelligent vers les équipes (dev vs UX vs support)
- Alimentation d'un dashboard produit en temps réel

> **Note production** : En contexte réel, ajouter une validation humaine pour les feedbacks haute priorité avant création automatique de tickets.

## 7. Comparaison des Approches

Récapitulatif des différentes méthodes pour obtenir des sorties structurées :

In [8]:
# Tableau comparatif
print("╔════════════════════════════════════════════════════════════════════════════════════╗")
print("║                      COMPARAISON DES APPROCHES JSON                                ║")
print("╠════════════════════════════════════════════════════════════════════════════════════╣")
print("║ Approche              │ JSON valide │ Structure garantie │ Type-safe │ Complexité ║")
print("╠════════════════════════════════════════════════════════════════════════════════════╣")
print("║ Prompt classique      │      ❌     │         ❌         │     ❌    │   Faible   ║")
print("║ JSON mode             │      ✅     │         ❌         │     ❌    │   Faible   ║")
print("║ JSON Schema manuel    │      ✅     │         ✅         │     ❌    │   Élevée   ║")
print("║ Pydantic + Structured │      ✅     │         ✅         │     ✅    │   Moyenne  ║")
print("╚════════════════════════════════════════════════════════════════════════════════════╝")
print("\nRecommandations :")
print("  • Prototypage rapide → JSON mode")
print("  • Production / APIs → Pydantic + Structured Outputs")
print("  • Insertion BDD → Pydantic + Structured Outputs")
print("  • Workflows complexes → Pydantic + Structured Outputs")

## Conclusion et Exercices

### Résumé

Les **Structured Outputs** d'OpenAI révolutionnent l'intégration LLM en production :

✅ **Fiabilité** : Garantie de conformité au schéma (plus de parsing fragile)  
✅ **Type Safety** : Pydantic apporte validation et auto-complétion  
✅ **Productivité** : Moins de code de validation manuelle  
✅ **Maintenabilité** : Schémas centralisés et réutilisables  

**Points clés à retenir :**
- `strict: True` est obligatoire pour Structured Outputs
- Pydantic simplifie drastiquement la définition de schémas
- Toujours valider avec `.model_validate_json()` pour type safety
- Gérer les erreurs API et de validation explicitement

---

### Exercices Suggérés

#### Exercice 1 : Extracteur d'Événements (Débutant)
Créer un modèle Pydantic pour extraire des événements d'un texte :
- Titre de l'événement
- Date et heure
- Lieu
- Participants (liste)

Tester avec : *"Réunion d'équipe le 15 mars 2025 à 14h30 dans la salle A. Participants : Alice, Bob, Charlie."*

#### Exercice 2 : Analyseur de Factures (Intermédiaire)
Définir un schéma pour extraire :
- Numéro de facture
- Date d'émission
- Fournisseur (nom, adresse)
- Articles (nom, quantité, prix unitaire, total)
- Montant total TTC

Générer une fausse facture avec GPT et extraire les données.

#### Exercice 3 : Générateur de Quiz (Avancé)
Créer un système qui :
1. Génère un quiz sur un sujet donné (5 questions)
2. Chaque question a 4 choix et une réponse correcte
3. Inclut une explication pour chaque réponse
4. Valide que la réponse correcte est bien dans les choix

Bonus : Ajouter des contraintes Pydantic (ex: `@field_validator`)

---

### Ressources

- [Documentation OpenAI Structured Outputs](https://platform.openai.com/docs/guides/structured-outputs)
- [Pydantic Documentation](https://docs.pydantic.dev/)
- [JSON Schema Reference](https://json-schema.org/understanding-json-schema/)
- [OpenAI Cookbook - Structured Outputs](https://cookbook.openai.com/examples/structured_outputs_intro)

---

**Prochaine étape :** Notebook 4 - Function Calling et Agents